In [ ]:
# Importación de librerias

import mlflow
from mlflow.models import infer_signature

import pandas as pd
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

## Información del dataset

Here’s the table in Markdown format:  

```markdown
| Feature                         | Type   |
|----------------------------------|--------|
| is_male                         | bool   |
| ethnicity                       | string |
| parental_level_of_education     | int8   |
| has_standard_lunch              | bool   |
| has_completed_preparation_test  | bool   |
| reading_score                   | int64  |
| writing_score                   | int64  |
| math_score                      | int64  |
```  

In [ ]:
# Carga del dataset